import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']

my_amazon_product_reviews = mydb['amazon_product_reviews']

In [60]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
#from nltk import word_tokenize
import string
from nltk.corpus import stopwords
            
            
class MySentences():
    def __init__(self):
        self.reviews_cursor = my_amazon_product_reviews.find()
        self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        self.stop = set(stopwords.words('english'))
        self.exclude = set(string.punctuation)
        self.lemma = WordNetLemmatizer()
 
    def __iter__(self):
        for review_item in self.reviews_cursor:
            reviewText = review_item["reviewText"]
            for sentence in self.sent_detector.tokenize(reviewText):
                #print(sentence)
                tokens = nltk.word_tokenize(sentence.lower())
                #tokens = word_tokenize(sentence.lower())
                #tokens = sentence.lower().split()
                normalized_tokens = [self.lemma.lemmatize(token) 
                                     for token in tokens 
                                     #if token not in self.stop and 
                                     token not in self.exclude
                                    ]
                print(normalized_tokens)
                yield normalized_tokens

In [ ]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
#from nltk import word_tokenize
import string
from nltk.corpus import stopwords
import gensim
            
            
class MySentences():
    def __init__(self):
        #self.reviews_cursor = my_amazon_product_reviews.find()
        self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        self.stop = set(stopwords.words('english'))
        self.exclude = set(string.punctuation)
        self.lemma = WordNetLemmatizer()
        self.i = 0
        

    def __iter__(self):

        r_i = 0
        review_item_iterator = iter(parse("../../download/reviews_Cell_Phones_and_Accessories.json.gz"))
        #for review_item in parse("../../download/reviews_Cell_Phones_and_Accessories.json.gz"): 
        for it in range(1000000):
            if it%10000==0:
                print("it="+str(it))
            review_item = next(review_item_iterator)
        #for review_item in self.reviews_cursor:
            r_i = r_i + 1
            if r_i%10000==0:
                print(str(r_i)+ " / " + str(self.i))
            reviewText = review_item["reviewText"]
            s_i = 0  
            for sentence in self.sent_detector.tokenize(reviewText):
                self.i = self.i + 1
                s_i = s_i + 1

                #print(sentence)
                tokens = nltk.word_tokenize(sentence.lower())
                #tokens = word_tokenize(sentence.lower())
                #tokens = sentence.lower().split()
                normalized_tokens = [self.lemma.lemmatize(token) 
                                     for token in tokens 
                                     #if token not in self.stop and 
                                     if token not in self.exclude
                                    ]
                #print(normalized_tokens)
                label = review_item["asin"] + "_" + review_item["reviewerID"] + "_" + str(s_i)
                yield gensim.models.doc2vec.LabeledSentence(words=normalized_tokens, tags=[label])
                
    

In [ ]:

sentences = MySentences()
model = gensim.models.Doc2Vec(sentences,min_count=100,max_vocab_size=1000000,workers=2)

it=0
10000 / 36312
it=10000
20000 / 94426
it=20000
30000 / 146636
it=30000
40000 / 197864
it=40000
50000 / 254984
it=50000
60000 / 307995
it=60000
70000 / 353570
it=70000
80000 / 400340
it=80000
90000 / 447233
it=90000
100000 / 492832
it=100000
110000 / 537302
it=110000
120000 / 584570
it=120000
130000 / 634276
it=130000
140000 / 674989
it=140000
150000 / 719040
it=150000
160000 / 766808
it=160000
170000 / 813538
it=170000
180000 / 861070
it=180000
190000 / 909602
it=190000
200000 / 955833
it=200000
210000 / 999524
it=210000
220000 / 1045994
it=220000
230000 / 1085179
it=230000
240000 / 1127464
it=240000
250000 / 1174366
it=250000
260000 / 1224181
it=260000
270000 / 1267179
it=270000
280000 / 1313303
it=280000
290000 / 1355170
it=290000
300000 / 1397379
it=300000
310000 / 1441219
it=310000
320000 / 1488466
it=320000
330000 / 1535052
it=330000
340000 / 1579727
it=340000
350000 / 1621663
it=350000
360000 / 1663572
it=360000
370000 / 1706038
it=370000
380000 / 1747383
it=380000
390000 / 1

In [ ]:
model.save('../../download/myamazon_doc2vec_model')